In [1]:
from keras.models import load_model
import pickle

import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import download
from keras.preprocessing.text import Tokenizer
from keras.backend import clear_session


import contractions

In [2]:
download('punkt')
download('stopwords')
download('punkt_tab')
download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Niko Amrullah
[nltk_data]     Hakam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Niko Amrullah
[nltk_data]     Hakam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Niko Amrullah
[nltk_data]     Hakam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Niko Amrullah
[nltk_data]     Hakam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
clear_session()

In [4]:
# Load the model
loaded_model = load_model('model')
print("Model loaded from 'model' directory")


Model loaded from 'model' directory


In [5]:
with open('tokenizer.pkl', 'rb') as f:
    loaded_tokenizer = pickle.load(f)
print("Tokenizer loaded from 'tokenizer.pkl'")

Tokenizer loaded from 'tokenizer.pkl'


In [6]:
# Initialize stopwords and stemmer
stpwds_en = list(set(stopwords.words('english')))

# init lemmitizer
lemmatizer = WordNetLemmatizer()

stpwds_en.append(["i", "to", "and", "my", "of", "the", "is", "have", "it", "in"])


def text_preprocessing(text):
    # Expand contractions
    text = contractions.fix(text)

    # Case folding
    text = text.lower()

    # Mention removal
    text = re.sub(r"@[A-Za-z0-9_]+", " ", text)

    # Hashtag removal
    text = re.sub(r"#[A-Za-z0-9_]+", " ", text)

    # Newline removal (\n)
    text = re.sub(r"\\n", " ", text)

    # Whitespace removal
    text = text.strip()

    # URL removal
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"www.\S+", " ", text)

    # Non-letter removal (retain apostrophes)
    text = re.sub(r"[^A-Za-z\s']", " ", text)

    # Tokenization
    tokens = word_tokenize(text)

    # Stopwords removal
    tokens = [word for word in tokens if word not in stpwds_en or word in {"not", "no", "never", "cannot", "still"}]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Combine tokens
    text = ' '.join(tokens)

    return text


In [7]:
# New data for inference
new_texts = ["I feel really anxious and stressed.", 
             "Life is going great, feeling very happy today. I have work to do but I am excited to do it.",
             "This is it. I am done. I am not doing this anymore. I am tired of this.",
             "I'm really happy today"]

# Preprocess text
preprocessed_texts = [text_preprocessing(text) for text in new_texts]

# Convert to sequences using the loaded tokenizer
sequences = loaded_tokenizer.texts_to_sequences(preprocessed_texts)

# Pad sequences to the same length as used during training
max_len = 180  # Use the same `max_len` from training
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

In [8]:
print(loaded_model.input_shape)

(None, 180)


In [9]:
# Make predictions
predictions = loaded_model.predict(padded_sequences)

# Output the predicted class (index of the highest probability)
predicted_classes = np.argmax(predictions, axis=1)

print("Predicted Classes:", predicted_classes)

1/1 [==============================] - 0s 312ms/step
Predicted Classes: [0 3 6 3]


Label Mapping: {'Anxiety': 0, 'Bipolar': 1, 'Depression': 2, 'Normal': 3, 'Personality disorder': 4, 'Stress': 5, 'Suicidal': 6}

In [10]:
# Correct class labels
class_labels = {
    0: 'Anxiety', 
    1: 'Bipolar', 
    2: 'Depression', 
    3: 'Normal', 
    4: 'Personality disorder', 
    5: 'Stress', 
    6: 'Suicidal'
}

# Map predicted classes to their labels
predicted_labels = [class_labels[cls] for cls in predicted_classes]
print("Predicted Labels:", predicted_labels)

Predicted Labels: ['Anxiety', 'Normal', 'Suicidal', 'Normal']


In [12]:
loaded_model.save('model.h5')

c:\ProgramData\anaconda3\envs\secondphase\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
